In [ ]:
import copclib as copc
import os
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
# Datei Einlesen:
reader = copc.FileReader("./daten/autzen.copc.laz")

# Copc Info und Las Header (für boxen)
copc_info = reader.copc_config.copc_info
las_header = reader.copc_config.las_header

# Bounding Box der Root node extrahieren: 

    # Root Voxelkey anlegen
root_key = copc.VoxelKey(0,0,0,0) # type: ignore

    # Root Entry (Node) laden (ohne Punkte)
root_node = reader.FindNode(root_key)

    # Bounding Box konstruieren aus VoxelKey und Las-Header
root_box = copc.Box(root_key, las_header)
print("Root Box:\n\txmin:{}\n\txmax:{}\n\n\tymin:{}\n\tymax:{}\n\n\tzmin:{}\n\tzmax:{}".format(root_box.x_min, root_box.x_max, root_box.y_min, root_box.y_max, root_box.z_min, root_box.z_max))


In [ ]:
# Überprüfung der Box
richtig_gerechnet = copc_info.center_x - copc_info.halfsize == root_box.x_min
print(f"{ copc_info.center_x - copc_info.halfsize }\n{ root_box.x_min }")
print(f"\nKonstruiert == selber ausgerechnet: {richtig_gerechnet}")

In [ ]:
# Alle Nodes auslesen und Bounding Boxen konstruieren

all_entries = reader.GetAllNodes()
boxes = []

for entry in all_entries:
    boxes.append(copc.Box(entry.key, las_header)) 

In [ ]:
from visuals import draw_boxes
from ray import Ray

# Erstelle Instanz eines Strahles
Strahl = Ray(np.array([640000, 855000, 5100]), np.array([-1, -3, -2]))

# Erstelle Eine Grafik mit mpl und den zwei Hilfsfunktionen
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

draw_boxes(boxes, ax)
Strahl.draw_ray(ax, 9000)